In [93]:
import pandas as pd
import numpy as np

from fill_and_map import map_econ_sdg, map_inds_sdg, fill_econ, fill_inds_trans

In [94]:
# get data
fpath = 'data_raw/SDG-DEA_indicators_V5_20200708.xlsx'
inds = fill_inds_trans(fpath)
inds_econ = fill_econ(fpath)
inds_map = map_inds_sdg(fpath)
econ_map = map_econ_sdg(fpath)

In [95]:
# separate indicators which will be used as undesired outputs, or not be used at all
use_inds = pd.read_excel('data_raw/use_inds.xlsx', usecols='A:D')
skip_inds = use_inds['ind_name'][use_inds['use']=='no']
undes_inds = use_inds['ind_name'][use_inds['issue']=='undesired']

In [96]:
all_sdgs = range(1,18)

# define set for countries
set_c = ', '.join((inds.index))

for sdg in all_sdgs:

    # define sets for inputs and outputs
    set_i = econ_map[sdg]
    des_out = list(set(inds_map[sdg]) - set(undes_inds) - set(skip_inds)) # set operation to get desired outputs
    undes_out = list(set(inds_map[sdg]) & set(undes_inds)) #  set operation to get undesired outputs

    if len(des_out) > 0:
        set_o = ', '.join(des_out)
    else: # placeholder set if empty
        set_o = 'dm'

    if len(undes_out) > 0:
        set_o_u = ', '.join(undes_out)
    else: # placeholder set if empty
        set_o_u = 'dm'

    # write sets to .inc file
    with open('data_input_to_gams/sets_sdg{0}.inc'.format(sdg), 'w') as text_file:
        set_string = 'set c / {0} /;\nset i / {1} /;\nset o / {2} /;\nset o_u / {3} /;'.format(set_c, set_i, set_o, set_o_u)
        text_file.write(set_string)

    # define parameters for inputs and outputs
    # placeholder column to use if no outputs or undesired outputs exist for an SDG
    dm = pd.DataFrame([1]*len(inds.index), index=inds.index, columns=['dm']) 

    # write inputs to .csv file
    inds_econ.loc[:,[econ_map[sdg]]].to_csv('data_input_to_gams/inputs_sdg{0}.csv'.format(sdg))

    # write outputs to .csv file
    if len(des_out) > 0:
        inds[des_out].to_csv('data_input_to_gams/outputs_sdg{0}.csv'.format(sdg))
    else:
        dm.to_csv('data_input_to_gams/outputs_sdg{0}.csv'.format(sdg))

    # write undesired outputs to .csv file
    if len(undes_out) > 0:
        inds[undes_out].to_csv('data_input_to_gams/undes_outputs_sdg{0}.csv'.format(sdg))
    else:
        dm.to_csv('data_input_to_gams/undes_outputs_sdg{0}.csv'.format(sdg))